In [120]:
import functools
import contextlib
import collections
from typing import Callable, Iterable, TypeVar, Mapping

import pyro
import torch  # noqa: F401

from chirho.counterfactual.handlers.selection import get_factual_indices
from chirho.indexed.ops import IndexSet, cond, gather, indices_of, scatter

S = TypeVar("S")
T = TypeVar("T")

import pyro
import chirho
import pyro.distributions as dist
import pyro.infer
import pytest
import torch
import pandas as pd

from chirho.counterfactual.handlers.counterfactual import (MultiWorldCounterfactual,
        Preemptions)
from chirho.counterfactual.handlers.explanation import (
    SearchForCause,
    consequent_differs,
    random_intervention,
    undo_split,
    uniform_proposal,
)
from chirho.counterfactual.ops import preempt, split
from chirho.indexed.ops import IndexSet, gather, indices_of
from chirho.observational.handlers.condition import Factors, condition
from chirho.interventional.ops import Intervention, intervene
from chirho.interventional.handlers import do




In [11]:
@contextlib.contextmanager
def ExplainCauses(
    antecedents: Mapping[str, Intervention[T]]
    | Mapping[str, pyro.distributions.constraints.Constraint],
    witnesses: Mapping[str, Intervention[T]] | Iterable[str],
    consequents: Mapping[str, Callable[[T], float | torch.Tensor]]
    | Iterable[str],
    *,
    antecedent_bias: float = 0.0,
    witness_bias: float = 0.0,
    consequent_eps: float = -1e8,
    antecedent_prefix: str = "__antecedent_",
    witness_prefix: str = "__witness_",
    consequent_prefix: str = "__consequent_",
):
    """
    Effect handler for causal explanation.

    :param antecedents: A mapping from antecedent names to interventions.
    :param witnesses: A mapping from witness names to interventions.
    :param consequents: A mapping from consequent names to factor functions.
    """
    if isinstance(
        next(iter(antecedents.values())),
        pyro.distributions.constraints.Constraint,
    ):
        antecedents = {
            a: random_intervention(s, name=f"{antecedent_prefix}_proposal_{a}")
            for a, s in antecedents.items()
        }

    if not isinstance(witnesses, collections.abc.Mapping):
        witnesses = {
            w: undo_split(antecedents=list(antecedents.keys()))
            for w in witnesses
        }

    if not isinstance(consequents, collections.abc.Mapping):
        consequents = {
            c: consequent_differs(
                antecedents=list(antecedents.keys()), eps=consequent_eps
            )
            for c in consequents
        }

    if len(consequents) == 0:
        raise ValueError("must have at least one consequent")

    if len(antecedents) == 0:
        raise ValueError("must have at least one antecedent")

    if set(consequents.keys()) & set(antecedents.keys()):
        raise ValueError(
            "consequents and possible antecedents must be disjoint"
        )

    if set(consequents.keys()) & set(witnesses.keys()):
        raise ValueError("consequents and possible witnesses must be disjoint")

    antecedent_handler = SearchForCause(
        actions=antecedents, bias=antecedent_bias, prefix=antecedent_prefix
    )
    witness_handler = Preemptions(
        actions=witnesses, bias=witness_bias, prefix=witness_prefix
    )
    consequent_handler = Factors(factors=consequents, prefix=consequent_prefix)

    with antecedent_handler, witness_handler, consequent_handler:
        with pyro.poutine.trace() as logging_tr:
            yield

### Stone-throwing

In [71]:
@pyro.infer.config_enumerate
def stones_model():        
    prob_sally_throws = pyro.sample("prob_sally_throws", dist.Beta(1, 1))
    prob_bill_throws = pyro.sample("prob_bill_throws", dist.Beta(1, 1))
    prob_sally_hits = pyro.sample("prob_sally_hits", dist.Beta(1, 1))
    prob_bill_hits = pyro.sample("prob_bill_hits", dist.Beta(1, 1))
    prob_bottle_shatters_if_sally = pyro.sample("prob_bottle_shatters_if_sally", dist.Beta(1, 1))
    prob_bottle_shatters_if_bill = pyro.sample("prob_bottle_shatters_if_bill", dist.Beta(1, 1))

    sally_throws = pyro.sample("sally_throws", dist.Bernoulli(prob_sally_throws))
    bill_throws = pyro.sample("bill_throws", dist.Bernoulli(prob_bill_throws))

#    print("stdtype", sally_throws.dtype)
#    print("btdtype", bill_throws.dtype)

    new_shp = torch.where(sally_throws == 1,prob_sally_hits, 0.0)

#    print("nsdtype", new_shp.dtype)

    sally_hits = pyro.sample("sally_hits",dist.Bernoulli(new_shp))

#    print("shdtype", sally_hits.dtype)
#    print("shdtype", sally_hits.dtype)
#    print( bill_throws.bool() & (~sally_hits.bool()))

    # new_bhp = torch.where(
    #         (
    #             bill_throws.bool()
    #             & (~sally_hits.bool())
    #         )
    #         == True,
    #         prob_bill_hits,
    #         torch.tensor(0.0),
    #     )

    new_bhp = torch.where(
        bill_throws.bool() & (~sally_hits.bool()),
        prob_bill_hits,
        torch.tensor(0.0),
    )

#   print("nbhdtype", new_bhp.dtype)

    bill_hits = pyro.sample("bill_hits", dist.Bernoulli(new_bhp))
    

    new_bsp = torch.where(bill_hits.bool(), prob_bottle_shatters_if_bill,
            torch.where(sally_hits.bool(),prob_bottle_shatters_if_sally,torch.tensor(0.0),),)

#    print("nbdsp", new_bsp.dtype)

    bottle_shatters = pyro.sample("bottle_shatters", dist.Bernoulli(new_bsp))

    return {"sally_throws": sally_throws, "bill_throws": bill_throws,  "sally_hits": sally_hits,
            "bill_hits": bill_hits,  "bottle_shatters": bottle_shatters,}

stones_model.nodes = ["sally_throws","bill_throws", "sally_hits", "bill_hits","bottle_shatters",]

In [81]:
observations = {"prob_sally_throws": 1.0, 
                "prob_bill_throws": 1.0,
                "prob_sally_hits": 1.0,
                "prob_bill_hits": 1.0,
                "prob_bottle_shatters_if_sally": 1.0,
                "prob_bottle_shatters_if_bill": 1.0,
                "sally_throws": 1.0, "bill_throws": 1.0}

observations_tensorized = {k: torch.as_tensor(v) for k, v in observations.items()}

antecedents = {"sally_throws": 0.0}
antencedent_bias = 0.1
witnesses = ["bill_throws", "bill_hits"]
consequents = "bottle_shatters"

In [133]:

with MultiWorldCounterfactual() as mwc:
    with ExplainCauses(antecedents = antecedents, antecedent_bias= antencedent_bias,
                        witnesses = witnesses,
                        consequents = consequents):
        with condition(data = observations_tensorized):
            with pyro.plate("sample", 15):
                with pyro.poutine.trace() as tr:
                    stones_model()

In [136]:
values_table = {}

trace = tr

#trace.trace.compute_log_prob()

nodes = trace.trace.nodes


print(nodes.keys())

with mwc:

    for antecedent_str in antecedents.keys():
            
        _int_ant = gather(
        nodes[antecedent_str]["value"],
        IndexSet(**{antecedent: {1} for antecedent in antecedents.keys()}),
        event_dim=0,)
 #      print(_int_ant.squeeze().tolist()

        values_table[f"{antecedent_str}_int"] = _int_ant.squeeze().tolist()
        _apr_ant = nodes[f"__antecedent_{antecedent_str}"]["value"]
        values_table[f"'apr_{antecedent_str}"] = _apr_ant.squeeze().tolist()
        function = nodes[f"__antecedent_{antecedent_str}"]["fn"]
        values_table[f"'apr_{antecedent_str}_lp"] = nodes[f"__antecedent_{antecedent_str}"]["fn"].log_prob(_apr_ant)


values_df = pd.DataFrame(values_table)
display(values_df)


odict_keys(['prob_sally_throws', 'prob_bill_throws', 'prob_sally_hits', 'prob_bill_hits', 'prob_bottle_shatters_if_sally', 'prob_bottle_shatters_if_bill', '__antecedent_sally_throws', 'sally_throws', '__witness_bill_throws', 'bill_throws', 'sally_hits', '__witness_bill_hits', 'bill_hits', 'bottle_shatters'])


,sally_throws_int,'apr_sally_throws,'apr_sally_throws_lp
0,0.0,0,-0.916291
1,1.0,1,-0.510826
2,1.0,1,-0.510826
3,1.0,1,-0.510826
4,0.0,0,-0.916291
5,1.0,1,-0.510826
6,1.0,1,-0.510826
7,1.0,1,-0.510826
8,0.0,0,-0.916291
9,0.0,0,-0.916291
